In [4]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
#image_dir = '/data/darknet/python/20190111_image/'
#xml_dir = '/data/darknet/python/20190111_xml/'
'''
image_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_Highway/VOC2007/JPEGImages/'
xml_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_Highway/VOC2007/Annotations/'

SEED_XML_DIR = xml_dir
SEED_IMG_DIR = image_dir
'''

GENE_OBJ_DIR = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/selected/obj/'
os.system('mkdir -p ' + GENE_OBJ_DIR)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_cls_from_xmlobj(obj_element):    
    return obj_element.find('name').text

def get_bbox_from_xmlobj(obj_element):
    xmlbox = obj_element.find('bndbox')
    return [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), 
            float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]

In [5]:
#classes = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']
classes = ['bus', 'truck']
for cls in classes:
    #image_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_Highway/VOC2007/JPEGImages/'
    #xml_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/VOCdevkit_Highway/VOC2007/Annotations/'
    image_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/selected/image/' + cls + '/'
    xml_dir = '/media/yyp/DATA/Public_Datasets/ZENITH_DATASETS/selected/xml/' + cls + '/'
    SEED_XML_DIR = xml_dir
    SEED_IMG_DIR = image_dir
    
    seed_xml_names = os.listdir(SEED_XML_DIR)
    seed_xml_names.sort()

    for xml_name in seed_xml_names:
        if(not xml_name.endswith('.xml')):
            continue
        img_name = SEED_IMG_DIR + xml_name[:-3] + 'jpg'
        xml_name = SEED_XML_DIR + xml_name
        #print(img_name)
        #print(xml_name)
        tree=ET.parse(xml_name)
        img = cv2.imread(img_name)    
        objs = get_obj_from_xml(xml_name)

        i = -1
        for obj_element in objs:
            bbox = get_bbox_from_xmlobj(obj_element)
            img_obj = get_obj_from_image_file(img_name, bbox)
            i = i+1
            cls = get_cls_from_xmlobj(obj_element)
            #if(cls not in classes):
                #continue
            cls_path = Path(GENE_OBJ_DIR + cls)
            if(not cls_path.is_dir()):
                os.system("mkdir -p " + GENE_OBJ_DIR + cls) # If use 'cls_path' : Can't convert 'PosixPath' object to str implicitly
            save_path_name = GENE_OBJ_DIR + cls + "/" + xml_name.split('/')[-1][:-4] + "_" + str(bbox[0]) + \
            "_" + str(bbox[1]) + "_" + str(bbox[2]) + "_" + str(bbox[3]) + '_' + str(i) + ".jpg"
            #print(save_path_name)
            cv2.imwrite(save_path_name, img_obj)
